# CV Assignment 1

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def LoadImage(image_path):
    im = Image.open(image_path).convert('L')  # Load and convert the image to grayscale
    return np.array(im)

def CalculateCo_occurrence(image_array):
    rows, cols = image_array.shape
    co_occurrence_matrix = np.zeros((256, 256), dtype=int)

    for i in range(rows):
        for j in range(cols):
            pixel_i = image_array[i, j]

            # Iterate over neighboring pixels within a defined window (e.g., 3x3 or 5x5)
            for m in range(max(0, i - 1), min(rows, i + 2)):
                for n in range(max(0, j - 1), min(cols, j + 2)):
                    if m == i and n == j:
                        continue  # Skip the central pixel

                    pixel_j = image_array[m, n]

                    # Increment the corresponding cell in the co-occurrence matrix
                    co_occurrence_matrix[pixel_i, pixel_j] += 1

    return co_occurrence_matrix

def CalculateContrast(co_occurrence_matrix):
    numerator = 0
    denominator = 0

    for i in range(256):
        for j in range(256):
            numerator += co_occurrence_matrix[i, j] * abs(i - j)
            denominator += co_occurrence_matrix[i, j]

    contrast = numerator / denominator
    return contrast

def CalculateHistogram(image_array):
    histogram, _ = np.histogram(image_array, bins=256, range=(0, 256))
    return histogram

def CalculateCumulativeHistogram(histogram):
    cumulative_histogram = np.cumsum(histogram)
    return cumulative_histogram

def GetColorAtPercentage(cumulative_histogram, percentage):
    total_pixels = cumulative_histogram[-1]
    target_pixel = int(percentage / 100 * total_pixels)

    low_intensity = np.argmax(cumulative_histogram >= target_pixel)
    high_intensity = 255 - np.argmax(np.flip(cumulative_histogram) >= (total_pixels - target_pixel))

    return low_intensity, high_intensity

def StretchContrast(image_array, low_intensity, high_intensity):
    stretched_image = (image_array - low_intensity) * (255.0 / (high_intensity - low_intensity))
    stretched_image = np.clip(stretched_image, 0, 255).astype(np.uint8)
    return stretched_image

def EqualizeHistogram(image_array, low_intensity, high_intensity):
    equalized_image = (image_array - low_intensity) * (255.0 / (high_intensity - low_intensity))
    equalized_image = np.clip(equalized_image, 0, 255).astype(np.uint8)
    return equalized_image

def GrayScaleTransformation(image_array, low_intensity1, high_intensity1, low_intensity2, high_intensity2):
    transformed_image = np.copy(image_array)
    mask1 = (image_array >= low_intensity1) & (image_array <= high_intensity1)
    mask2 = (image_array >= low_intensity2) & (image_array <= high_intensity2)
    transformed_image[mask1] = 0
    transformed_image[mask2] = 255
    return transformed_image

def main():
    image_path = "image.png"
    image_array = LoadImage(image_path)

    # Calculate co-occurrence matrix and contrast
    co_occurrence_matrix = CalculateCo_occurrence(image_array)
    contrast = CalculateContrast(co_occurrence_matrix)
    print(f"Contrast: {contrast}")

    # Calculate image histogram and cumulative histogram
    histogram = CalculateHistogram(image_array)
    cumulative_histogram = CalculateCumulativeHistogram(histogram)

    # Get color intensities at specific percentages
    low_intensity, high_intensity = GetColorAtPercentage(cumulative_histogram, 5)
    print(f"5% Color Intensities: Low - {low_intensity}, High - {high_intensity}")

    # Apply contrast stretching
    stretched_image = StretchContrast(image_array, low_intensity, high_intensity)

    # Apply histogram equalization
    equalized_image = EqualizeHistogram(image_array, low_intensity, high_intensity)

    # Apply gray-scale transformation
    transformed_image = GrayScaleTransformation(image_array, low_intensity, high_intensity, 128, 192)

    # Display the images
    plt.figure(figsize=(12, 12))
    plt.subplot(2, 3, 1), plt.imshow(image_array, cmap='gray'), plt.title('Original Image')
    plt.subplot(2, 3, 2), plt.imshow(stretched_image, cmap='gray'), plt.title('Contrast Stretching')
    plt.subplot(2, 3, 3), plt.imshow(equalized_image, cmap='gray'), plt.title('Histogram Equalization')
    plt.subplot(2, 3, 4), plt.hist(image_array.ravel(), bins=256, range=(0, 256), color='k', alpha=0.7), plt.title('Original Histogram')
    plt.subplot(2, 3, 5), plt.hist(stretched_image.ravel(), bins=256, range=(0, 256), color='k', alpha=0.7), plt.title('Stretched Histogram')
    plt.subplot(2, 3, 6), plt.hist(equalized_image.ravel(), bins=256, range=(0, 256), color='k', alpha=0.7), plt.title('Equalized Histogram')
    plt.show()

if __name__ == '__main__':
    main()